In [35]:
import random
import math
class Node:
    def __init__(self, weight):
        self.weight = weight
        self.bias = random.random() * 10
        
    def giveWeight(self):
        return self.weight 
    
    def giveBias(self):
        return self.bias
    
    def calculate(self, values):
        val = 0
        for i in range(len(values)):
            numToAdd = values[i]
            val += (numToAdd * self.weight)
        
        val += self.bias
        
        if val > 33:
            val = 32
        elif val < -33:
            val = -32
            
        activation = 1/(1 + pow(2.718, val))
        
        if activation <= 0.5:
            return activation
        return -activation
    
    def change(self, direction, weight):
        if (direction == "pos"):
            self.weight += weight
        else:
            self.weight -= weight
    
    def changeB(self, direction, bias):
        if (direction == "pos"):
            self.bias += bias
        else:
            self.bias -= bias
    
    def setWeight(self, newWeight):
        self.weight = newWeight
    
    def setBias(self, newBias):
        self.bias = newBias
    
class Layer:
    layer = []
    def __init__(self, numNodes):
        self.layer = []
        self.numNodes = numNodes
        for i in range(numNodes):
            self.layer.append(Node(random.random()))
    
    def forwardProp(self, values):
        ret = []
        for i in range(self.numNodes):
            ret.append(self.layer[i].calculate(values))
        return ret
    
    def inputLayer(self, values):
        ret = []
        for i in range(len(values)):
            temp = []
            temp.append(values[i])
            ret.append(self.layer[i].calculate(temp))
        return ret
    
    def weightsChange(self, direction):
        for i in range(len(self.layer)):
            if direction == "pos":
                self.layer[i].change(0.01, 0)
            else:
                self.layer[i].change(-0.01, 0)
    
class Network:
    network = []
    def __init__(self, depth, inputs, outputs):
        self.network = []
        self.depth = depth
        self.inputs = inputs
        self.outputs = outputs
        self.iterationCap = 100
        self.changeBy = 0.5
        flattenAmt = int(inputs/2)
                
        self.network.append(Layer(inputs))
        
        self.network.append(Layer(flattenAmt))
        
        for i in range(depth):
            count = 0
            if i < int(depth/2):
                self.network.append(Layer(flattenAmt + i))
            else:
                if (count < outputs + 1):
                    self.network.append(Layer(outputs - count))
                    count += 1
            
        self.network.append(Layer(outputs))
        
    def setIterationCap(self, newIterationCap):
        self.iterationCap = newIterationCap
        
    def setChangeBy(self, newChangeBy):
        self.changeBy = newChangeBy
    
    def setWeights(self, weightlist):
        for a in range(len(self.network)):
            for b in range(len(self.network[a].layer)):
                self.network[a].layer[b].setWeight(weightlist[a][b])
                
    def setBiases(self, biaslist):
        for a in range(len(self.network)):
            for b in range(len(self.network[a].layer)):
                self.network[a].layer[b].setBias(weightlist[a][b])
                
    def getWeights(self):
        temp = []
        for a in range(len(self.network)):
            microarray = []
            for b in range(len(self.network[a].layer)):
                microarray.append(self.network[a].layer[b].giveWeight())
            temp.append(microarray)
        return temp
    
    def getBiases(self):
        temp = []
        for a in range(len(self.network)):
            microarray = []
            for b in range(len(self.network[a].layer)):
                microarray.append(self.network[a].layer[b].giveBias())
            temp.append(microarray)
        return temp
                
    def printWeights(self):
        for i in range(len(self.network)):
            print("-----")
            for j in range(len(self.network[i].layer)):
                print("weight:", self.network[i].layer[j].weight)
                print()
                
    def printBiases(self):
        for i in range(len(self.network)):
            print("------")
            for j in range(len(self.network[i].layer)):
                print("bias:", self.network[i].layer[j].bias)
                print()
                
    def forwardPropagate(self, userInput):
        #convert user input
        converted = []
        for i in range(len(userInput)):
            for j in range(len(userInput[i])):
                converted.append(userInput[i][j] * (i + j + 1))
        ret = self.network[0].inputLayer(converted)
        for i in range(1, self.depth):
            ret = self.network[i].forwardProp(ret)
        return ret

    def calculateError(self, userInputs, desiredResults):
        error = 0
        for i in range(len(userInputs)):
            temp = self.forwardPropagate(userInputs[i])
            if desiredResults[i] != temp:
                error += (abs(desiredResults[i][0] - temp[0]) + abs(desiredResults[i][1] - temp[1]))
        return error/(len(desiredResults) * len(desiredResults[0]))
    
    def backPropagate(self, userInputs, desiredResults, percentAccurate):    
        iterationCap = self.iterationCap
        changeBy = self.changeBy
        
        layerOn = 0
        error = self.calculateError(userInputs, desiredResults)
        initialError = error
        while (layerOn < len(self.network)):
            nodeOn = 0

            while (nodeOn < len(self.network[layerOn].layer)):
                #forward until not better
                improved = True
                count = 0
                countI = 0
                for j in range(iterationCap):
                    self.network[layerOn].layer[nodeOn].change("pos", changeBy)
                    newerror = self.calculateError(userInputs, desiredResults)
                    count += 1

                    if (newerror*newerror) < (error * error):
                        error = newerror
                        improved = True
                        countI = count
                    else:
                        improved = False
                if improved == False:
                    j = iterationCap
                    while (j > countI):
                        self.network[layerOn].layer[nodeOn].change("neg", changeBy)
                        j -= 1

                #backward until not better
                improved = True
                count = 0
                countI = 0
                for j in range(iterationCap):
                    self.network[layerOn].layer[nodeOn].change("neg", changeBy)
                    newerror = self.calculateError(userInputs, desiredResults)
                    count += 1

                    if (newerror*newerror) < (error * error):
                        error = newerror
                        improved = True
                        countI = count
                    else:
                        improved = False
                if improved == False:
                    j = iterationCap
                    while (j > countI):
                        self.network[layerOn].layer[nodeOn].change("pos", changeBy)
                        j -= 1

                nodeOn += 1
            layerOn += 1

        ########
        iterationCap = self.iterationCap
        changeBy = self.changeBy

        layerOn = 0
        error = self.calculateError(userInputs, desiredResults)
        initialError = error
        while (layerOn < len(self.network)):
            nodeOn = 0

            while (nodeOn < len(self.network[layerOn].layer)):
                #forward until not better
                improved = True
                count = 0
                countI = 0
                for j in range(iterationCap):
                    self.network[layerOn].layer[nodeOn].changeB("pos", changeBy)
                    newerror = self.calculateError(userInputs, desiredResults)
                    count += 1

                    if (newerror*newerror) < (error * error):
                        error = newerror
                        improved = True
                        countI = count
                    else:
                        improved = False
                if improved == False:
                    j = iterationCap
                    while (j > countI):
                        self.network[layerOn].layer[nodeOn].changeB("neg", changeBy)
                        j -= 1

                #backward until not better
                improved = True
                count = 0
                countI = 0
                for j in range(iterationCap):
                    self.network[layerOn].layer[nodeOn].changeB("neg", changeBy)
                    newerror = self.calculateError(userInputs, desiredResults)
                    count += 1
                    
                    if (newerror*newerror) < (error * error):
                        error = newerror
                        improved = True
                        countI = count
                    else:
                        improved = False
                if improved == False:
                    j = iterationCap
                    while (j > countI):
                        self.network[layerOn].layer[nodeOn].changeB("pos", changeBy)
                        j -= 1

                nodeOn += 1
            layerOn += 1
        
                
    def userTest(self, userInput):
        ret = self.forwardPropagate(userInput)
        print(ret[0], ret[1])

        if (ret[0] < ret[1]):
            print("this is a square")
        elif (ret[0] > ret[1]):
            print("this is not a square")
        else:
            print("this was not supposed to happen")
            
listInputs = []
answerInputs = []

answerInputs.append([0, 1])
arr1 = [[1, 1, 0],
        [1, 1, 0],
        [0, 0, 0]]
listInputs.append(arr1)

answerInputs.append([0, 1])
arr2 = [[0, 0, 0],
        [0, 1, 1],
        [0, 1, 1]]
listInputs.append(arr2)

answerInputs.append([1, 0])
arr3 = [[1, 1, 0],
        [1, 1, 0],
        [1, 1, 0]]
listInputs.append(arr3)

answerInputs.append([1, 0])
arr4 = [[0, 0, 1],
        [1, 1, 1],
        [0, 1, 0]]
listInputs.append(arr4)

answerInputs.append([1, 0])
arr5 = [[1, 0, 0],
        [1, 1, 0],
        [0, 0, 1]]
listInputs.append(arr5)

answerInputs.append([1, 0])
arr6 = [[1, 1, 1],
        [1, 0, 0],
        [0, 1, 0]]
listInputs.append(arr6)

answerInputs.append([1, 0])
arr7 = [[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]]
listInputs.append(arr7)

answerInputs.append([0, 1])
arr8 = [[0, 0, 0],
        [1, 1, 0],
        [1, 1, 0]]
listInputs.append(arr8)

answerInputs.append([0, 1])
arr9 = [[1, 1, 0],
        [1, 1, 0],
        [0, 0, 0]]
listInputs.append(arr9)

answerInputs.append([0, 1])
arr10 = [[0, 0, 0],
         [0, 1, 1],
         [0, 1, 1]]
listInputs.append(arr10)

##############################
trainset1 = [[1, 0, 0],
             [1, 1, 1],
             [1, 1, 0]]
trainset2 = [[1, 0, 0],
             [1, 0, 1],
             [0, 0, 1]]
trainset3 = [[0, 0, 0],
             [0, 1, 1],
             [0, 1, 1]]
trainset4 = [[0, 1, 1],
             [0, 1, 1],
             [0, 0, 0]]
trainset5 = [[0, 0, 0],
             [0, 0, 0],
             [0, 0, 0]]

neuralNet = Network(6, 9, 2)

print("this is error before: ", neuralNet.calculateError(listInputs, answerInputs))

neuralNet.setIterationCap(100)
neuralNet.setChangeBy(.61803398875/1.61803398875) #best of .25 error

neuralNet.backPropagate(listInputs, answerInputs, 0.9)
print("this is error after: ", neuralNet.calculateError(listInputs, answerInputs))

neuralNet.userTest(trainset1)
neuralNet.userTest(trainset2)
neuralNet.userTest(trainset3)
neuralNet.userTest(trainset4)

neuralNet.setWeights(neuralNet.getWeights())

this is error before:  0.4999999765211759
this is error after:  0.30149317175901136
0.4072599541376298 0.000156153969552847
this is not a square
0.4058565264731931 0.00015716417158779004
this is not a square
0.0002612096276359491 0.49161506156045476
this is a square
0.00026116694773181887 0.49166035491006216
this is a square


In [36]:
for abcdefg in range(3):
    neuralNet.backPropagate(listInputs, answerInputs, 0.9)

neuralNet.userTest(trainset1)
neuralNet.userTest(trainset2)
neuralNet.userTest(trainset3)
neuralNet.userTest(trainset4)
print("this is error after: ", neuralNet.calculateError(listInputs, answerInputs))

0.4247711170825832 0.00014416068371079607
this is not a square
0.16916849910293688 0.0006011754339758908
this is not a square
0.00025344686857350874 0.4999788930268122
this is a square
0.00025386537151846956 0.49952145100541856
this is a square
this is error after:  0.26856438235356894


In [39]:
neuralNet.userTest(trainset1)
for i in range(len(trainset1)):
    print(trainset1[i])

neuralNet.userTest(trainset2)
for i in range(len(trainset2)):
    print(trainset2[i])
    
neuralNet.userTest(trainset3)
for i in range(len(trainset3)):
    print(trainset3[i])
    
neuralNet.userTest(trainset4)
for i in range(len(trainset4)):
    print(trainset4[i])
    
neuralNet.userTest(trainset5)
for i in range(len(trainset5)):
    print(trainset5[i])

0.4247711170825832 0.00014416068371079607
this is not a square
[1, 0, 0]
[1, 1, 1]
[1, 1, 0]
0.16916849910293688 0.0006011754339758908
this is not a square
[1, 0, 0]
[1, 0, 1]
[0, 0, 1]
0.00025344686857350874 0.4999788930268122
this is a square
[0, 0, 0]
[0, 1, 1]
[0, 1, 1]
0.00025386537151846956 0.49952145100541856
this is a square
[0, 1, 1]
[0, 1, 1]
[0, 0, 0]
0.00025342910927191156 0.49999832142510603
this is a square
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]


In [40]:
print("this is error after: ", neuralNet.calculateError(listInputs, answerInputs))

this is error after:  0.26856438235356894


In [44]:
neuralNet.userTest([[0, 0, 0],
                    [0, 1, 0],
                    [0, 0, 0]])
print("0 0 0")
print("0 1 0")
print("0 0 0")
print()

neuralNet.userTest([[1, 1, 1],
                    [1, 1, 1],
                    [1, 1, 1]])
print("1 1 1")
print("1 1 1")
print("1 1 1")
print()

neuralNet.userTest([[1, 1, 1],
                    [1, 0, 1],
                    [1, 1, 1]])
print("1 1 1")
print("1 0 1")
print("1 1 1")
print()

neuralNet.userTest([[0, 0, 0],
                    [1, 1, 0],
                    [1, 1, 0]])
print("0 0 0")
print("1 1 0")
print("1 1 0")
print()

neuralNet.userTest([[1, 1, 0],
                    [1, 1, 0],
                    [0, 0, 0]])
print("1 1 0")
print("1 1 0")
print("0 0 0")
print()

neuralNet.userTest([[0, 1, 1],
                    [0, 1, 1],
                    [0, 0, 0]])
print("0 1 1")
print("0 1 1")
print("0 0 0")
print()

neuralNet.userTest([[0, 0, 0],
                    [0, 1, 1],
                    [0, 1, 1]])
print("0 0 0")
print("0 1 1")
print("0 1 1")
print()


0.00025342891747790017 0.49999853125218563
this is a square
0 0 0
0 1 0
0 0 0

0.42472218575990667 0.00014419269203960075
this is not a square
1 1 1
1 1 1
1 1 1

0.4247221185449635 0.0001441927360136744
this is not a square
1 1 1
1 0 1
1 1 1

0.00025566675334416666 0.4975610488826702
this is a square
0 0 0
1 1 0
1 1 0

0.0002557008375809605 0.49752408958721533
this is a square
1 1 0
1 1 0
0 0 0

0.00025386537151846956 0.49952145100541856
this is a square
0 1 1
0 1 1
0 0 0

0.00025344686857350874 0.4999788930268122
this is a square
0 0 0
0 1 1
0 1 1



In [45]:
neuralNet.printWeights()
neuralNet.printBiases()

-----
weight: -16.141364767171957

weight: -4.894371948769771

weight: -1.8970577051791875

weight: 10.455136370624045

weight: -0.21062833168517264

weight: 0.9250840432631413

weight: -9.557101856653066

weight: 0.9312461204788895

weight: 3.8640127516850455

-----
weight: -35.12521896490476

weight: -15.208286325554091

weight: -12.364410703433151

weight: -6.434138964408107

-----
weight: -21.843114627672147

weight: 0.5169641225331936

weight: -36.172676558825

weight: 1.515073825625276

-----
weight: 9.747230575896628

weight: -0.34955674728695135

weight: -36.45153402734712

weight: 29.37760373760172

weight: 0.9637748126031489

-----
weight: -35.24845840516565

weight: -32.65777985009771

weight: 0.7447304931789753

weight: 9.930214100950638

weight: 0.1868435960365905

weight: 7.410564768150691

-----
weight: 2.286056099985248

weight: -2.534659489848555

-----
weight: 0.49183867569598533

weight: 0.48861205971025384

-----
weight: 0.25653438362499037

weight: 0.50605729191087